In [1]:
#!/usr/bin/env python
# coding: utf-8
get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold

import gc

from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings(action='ignore')


import torch
print(torch.__version__)
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchsummary import summary

print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
device

/home/aiden/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


2.3.2
1.3.1
GeForce RTX 2070 SUPER
True


device(type='cuda', index=0)

In [2]:
# import torch
# import torchtext
# from torchtext.datasets import text_classification
# NGRAMS = 2
# import os
# if not os.path.isdir('./.data'):
#     os.mkdir('./.data')
# train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
#     root='./.data', ngrams=NGRAMS, vocab=None)
# BATCH_SIZE = 16

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# train_dataset.

# import torch.nn as nn
# import torch.nn.functional as F

# class TextSentiment(nn.Module):
#     def __init__(self, vocab_size, embed_dim, num_class):
#         super().__init__()
#         self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
#         self.fc = nn.Linear(embed_dim, num_class)
#         self.init_weights()

#     def init_weights(self):
#         initrange = 0.5
#         self.embedding.weight.data.uniform_(-initrange, initrange)
#         self.fc.weight.data.uniform_(-initrange, initrange)
#         self.fc.bias.data.zero_()

#     def forward(self, text, offsets):
#         embedded = self.embedding(text, offsets)
#         return self.fc(embedded)

# VOCAB_SIZE = len(train_dataset.get_vocab())
# EMBED_DIM = 32
# NUN_CLASS = len(train_dataset.get_labels())
# model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

#### Model

In [3]:
class LRModel(torch.nn.Module):
    def __init__(self, input_size):
        super(LRModel,self).__init__()

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 2), 
#             torch.nn.Sigmoid()
        )
        
    def forward(self, x):        
        return self.model(x) 

In [4]:
class NNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super(NNModel,self).__init__()
        relu = torch.nn.ReLU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.model = torch.nn.Sequential(
#             torch.nn.Linear(input_size, 4096), relu, torch.nn.BatchNorm1d(4096), dropout,
#             torch.nn.Linear(4096, 2048), relu, torch.nn.BatchNorm1d(2048), dropout,
            
            torch.nn.Linear(input_size, 2048), relu, torch.nn.BatchNorm1d(2048), dropout,
            torch.nn.Linear(2048, 1024), relu, torch.nn.BatchNorm1d(1024), dropout,
            
#             torch.nn.Linear(input_size, 1024), relu, torch.nn.BatchNorm1d(1024), dropout, 

            torch.nn.Linear(1024, 512), relu, torch.nn.BatchNorm1d(512), dropout,
            torch.nn.Linear(512, 512), relu, torch.nn.BatchNorm1d(512), dropout,
            torch.nn.Linear(512, 256), relu, torch.nn.BatchNorm1d(256), dropout,
            torch.nn.Linear(256, 128), relu, torch.nn.BatchNorm1d(128), dropout,
            torch.nn.Linear(128, 2), 
#             torch.nn.Sigmoid()
        )
        
    def forward(self, x):        
        return self.model(x) 

#### Dataset

In [5]:
class KBDataset(Dataset):
    def __init__(self, df, y_col):
#         print(df.shape)
#         print(df.info())
        
        self.columns = [c for c in df.columns if c not in [y_col]]
        self.X = df[self.columns].values
        self.y = pd.get_dummies(df[y_col].astype(int), prefix=y_col).values

    def __len__(self):
        return len(self.X)
    
    def get_feature_names(self):
        return self.columns

    def __getitem__(self, idx):
        return self.X[idx].astype(np.float32), self.y[idx].astype(np.float32)

#### eval_summary

In [6]:
def eval_summary(y_true, y_score, cut_off=0.5):
    if len(y_true) == 0 or len(y_score) == 0:
        return 'zero length'
    if len(y_true) != len(y_score):
        return 'diff length'
    
    y_pred = y_score.copy()
    y_pred[y_pred > cut_off] = 1
    y_pred[y_pred <= cut_off] = 0

    eval_dict = {}
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score, pos_label=1)
    
    eval_dict['auc'] = metrics.auc(fpr, tpr)
    eval_dict['confusion_matrix'] = metrics.confusion_matrix(y_true, y_pred)
    
    pre, rec, _, _ = metrics.precision_recall_fscore_support(y_true, y_pred, pos_label=1)
    eval_dict['precision'] = pre[1]
    eval_dict['recall'] = rec[1]
    
    return eval_dict

#### train/test_torch

In [7]:
from torch.utils.data import DataLoader
step = 4
num_workers = 6

def train_torch(dataset):
    model.train()
    loss = 0
    acc = 0
    data_loader = DataLoader(dataset=dataset,
#                           batch_size=100000,
#                         batch_size=int(train_size * 0.7),
                          batch_size=len(dataset) // step,
#                           batch_size=10000,
                          shuffle=True,
                          num_workers=num_workers,
                         drop_last=True
                         )
    for i, data in enumerate(data_loader):
#     for i, data in tqdm_notebook(enumerate(train_loader), total=len(train_loader), desc = 'epoch{}_batch'.format(e)):
#         print(e, i)
        X_batch, y_batch = data
        
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch)
        
        loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
        
        del X_batch, y_batch, y_pred
        gc.collect()

    return loss / len(dataset), acc / len(dataset)


def test_torch(dataset):
    model.eval()
    loss = 0
    acc = 0
    
    data_loader = DataLoader(dataset=dataset,
                          batch_size=len(dataset) // step,
                          shuffle=False,
                          num_workers=num_workers,
                          drop_last=True
                         ) 
    
    for i, data in enumerate(data_loader):
        X_batch, y_batch = data
        
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        with torch.no_grad():
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss += loss.item()
            acc += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
            
            y_pred = torch.sigmoid(y_pred)
            
            del X_batch, y_batch, y_pred
            
    return loss / len(dataset), acc / len(dataset)


def pred_torch(dataset):
    model.eval()
    loss = 0
    acc = 0
    
    y_true_list = []
    y_score_list = []

    data_loader = DataLoader(dataset=dataset,
                          batch_size=len(dataset) // step,
                          shuffle=False,
                          num_workers=1,
                          drop_last=True
                         ) 
    
    for i, data in enumerate(data_loader):
        X_batch, y_batch = data
        
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        y_true = y_batch
        y_true_list.append(y_true[:, 1].cpu().detach().numpy())

        with torch.no_grad():
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss += loss.item()
            acc += (y_pred.argmax(1) == y_batch.argmax(1)).sum().item()
            
            y_pred = torch.sigmoid(y_pred)
            y_score_list.append(y_pred[:, 1].cpu().detach().numpy())
            
            del X_batch, y_batch, y_true, y_pred
            
    return loss / len(dataset), acc / len(dataset), np.concatenate(y_true_list, axis=0), np.concatenate(y_score_list, axis=0)


#### Load Data

In [8]:
# merged_ts = '20191230T014439_8180'
# merged_ts = '20191229T155539'
# merged_ts = '20191231T113708_5499'

merged_ts = '20191231T165424_6099'


train_path = 'data/df_merged_{}_train.pkl'.format(merged_ts)
test_path = 'data/df_merged_{}_test.pkl'.format(merged_ts)

df_model = joblib.load(train_path)  
df_model = df_model.reset_index()
print('model_set\n', df_model['smishing'].value_counts())
df_test = joblib.load(test_path) 

idx_cols = ['smishing', 'id', 'index']
fea_cols = [c for c in df_model.columns if c not in idx_cols]#[:500]
input_size = len(fea_cols)

x_test = torch.Tensor(df_test[fea_cols].values).to(device)

model_set
 0    277242
1     18703
Name: smishing, dtype: int64


In [9]:
df_model[fea_cols]

,fea__text_len,fea__noun_cnt,cnt_0000,cnt_0001,cnt_0002,cnt_0003,cnt_0004,cnt_0005,cnt_0006,cnt_0007,...,tfidf_pos_word_22_0490,tfidf_pos_word_22_0491,tfidf_pos_word_22_0492,tfidf_pos_word_22_0493,tfidf_pos_word_22_0494,tfidf_pos_word_22_0495,tfidf_pos_word_22_0496,tfidf_pos_word_22_0497,tfidf_pos_word_22_0498,tfidf_pos_word_22_0499
0,24.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,37.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,81.0,12.0,11.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,174.0,29.0,34.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,40.0,4.0,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295940,168.0,18.0,32.0,1.0,0.0,0.0,0.0,0.0,4.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
295941,33.0,9.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
295942,97.0,15.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
295943,66.0,15.0,9.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Load Model

In [10]:
# model = NNModel(input_size=input_size, dropout_probability=0.7).to(device)
# epoch = 1
# print(summary(model, (input_size, )))

#### Training

In [11]:
# import EarlyStopping
from torchtools import EarlyStopping

In [ ]:
[df_test.drop(c, axis=1, inplace=True) for c in df_test.columns if 'smishing_' in c]

skf = StratifiedKFold(n_splits=5, random_state=8405)

for cv, index in enumerate(skf.split(df_model[fea_cols], df_model['smishing'])):
    train_index, valid_index = index
    
    print(len(train_index), len(valid_index))
    print('\nCV', cv)
    model = NNModel(input_size=input_size, dropout_probability=0.7).to(device)
#     model =  LRModel(input_size=input_size).to(device)

    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=15, verbose=True)

    epoch = 1
    if cv == 0:
        print(summary(model, (input_size, )))
    
    pos_weight = torch.Tensor([1., 10.,])
#     pos_weight = torch.Tensor([1., 1.,])
    criterion = torch.nn.BCEWithLogitsLoss(reduction='sum', pos_weight=pos_weight).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
    print(model_ts)
    print('Epoch:', epoch)

    N_EPOCHS = 100
    is_summary = True
    for e in tqdm_notebook(range(epoch, epoch + N_EPOCHS), total=N_EPOCHS, desc = 'CV {} Epoch'.format(cv)):
        start_time = time.time()
        train_set = KBDataset(df_model.loc[train_index, fea_cols + ['smishing']], 'smishing')
        valid_set = KBDataset(df_model.loc[valid_index, fea_cols + ['smishing']], 'smishing')

        train_loss, train_acc = train_torch(train_set)
        valid_loss, valid_acc = test_torch(valid_set)
        print('CV {} Epoch {}\n\tTrain loss: {}\tValid loss: {}\t{}'.format(cv, e, train_loss, valid_loss, train_loss / valid_loss))
        
        early_stopping(valid_loss, model)
        
        
        if early_stopping.counter == 0:
            if is_summary:
                _, _, y_true, y_score = pred_torch(valid_set)            
                print('\t', eval_summary(y_true, y_score, cut_off=0.5))
                is_summary = False
        else:
            is_summary = True
            

        if early_stopping.early_stop:
            print("\tEarly stopping epoch {}, valid loss {}".format(e, valid_loss))
            break
    
        del train_set, valid_set
        gc.collect()
        
        epoch = e + 1
    
    # load the last checkpoint with the best model
    model.load_state_dict(torch.load('checkpoint.pt'))
    
    valid_set = KBDataset(df_model.loc[valid_index, fea_cols + ['smishing']], 'smishing')
    _, _, y_true, y_score = pred_torch(valid_set)            
    print('\t', eval_summary(y_true, y_score, cut_off=0.5))

    train_set = KBDataset(df_model.loc[train_index, fea_cols + ['smishing']], 'smishing')
    _, _, y_true, y_score = pred_torch(train_set)            
    print('END CV {} eval summary (train)\n'.format(cv), eval_summary(y_true, y_score, cut_off=0.5))

    torch.save(model.state_dict(), 'model/{}_{}_{}.model'.format(model_ts, cv, epoch-1))
    
    model.eval()
    pred_col = 'smishing_{}'.format(cv)
    df_test[pred_col] = torch.sigmoid(model(x_test))[:, 1].cpu().detach().numpy()
    df_test[[pred_col]].to_csv('submit/{}_{}_nn.csv'.format(model_ts, pred_col), index=True)
    
#     break

236755 59190

CV 0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 2048]      12,492,800
              ReLU-2                 [-1, 2048]               0
       BatchNorm1d-3                 [-1, 2048]           4,096
           Dropout-4                 [-1, 2048]               0
            Linear-5                 [-1, 1024]       2,098,176
              ReLU-6                 [-1, 1024]               0
       BatchNorm1d-7                 [-1, 1024]           2,048
           Dropout-8                 [-1, 1024]               0
            Linear-9                  [-1, 512]         524,800
             ReLU-10                  [-1, 512]               0
      BatchNorm1d-11                  [-1, 512]           1,024
          Dropout-12                  [-1, 512]               0
           Linear-13                  [-1, 512]         262,656
             ReLU-14

CV 0 Epoch 1
	Train loss: 0.7742999196052551	Valid loss: 0.6042466163635254	1.2814302444458008
Validation loss decreased (inf --> 0.604247).  Saving model ...
	 {'auc': 0.9931941801899445, 'confusion_matrix': array([[47568,  7881],
       [    6,  3733]]), 'precision': 0.3214224212157741, 'recall': 0.9983952928590533}
CV 0 Epoch 2
	Train loss: 0.6421478390693665	Valid loss: 0.5045629143714905	1.2726813554763794
Validation loss decreased (0.604247 --> 0.504563).  Saving model ...
CV 0 Epoch 3
	Train loss: 0.527597188949585	Valid loss: 0.32794803380966187	1.6087828874588013
Validation loss decreased (0.504563 --> 0.327948).  Saving model ...
CV 0 Epoch 4
	Train loss: 0.4310115575790405	Valid loss: 0.28785645961761475	1.497314214706421
Validation loss decreased (0.327948 --> 0.287856).  Saving model ...


#### Predict Train

In [ ]:
df = pd.Series(y_score)
df.hist(bins=100, figsize=(20, 5))
(df * 10).astype(int).value_counts(sort=False)

In [ ]:
# df_model[(y_score <= 0.5) & (y_true == 1)]['text']

In [ ]:
# df_model[(y_score > 0.5) & (y_true == 0)]['text']

#### Predict Test

In [ ]:
pred_cols = [c for c in df_test.columns if 'smishing_' in c]
print(len(pred_cols))
df_test['pred_max'] = df_test[pred_cols].max(axis=1)
df_test['pred_min'] = df_test[pred_cols].min(axis=1)
df_test['pred_mean'] = df_test[pred_cols].mean(axis=1)
df_test['pred_std'] = df_test[pred_cols].std(axis=1)

print(df_test['pred_std'].max(), df_test['pred_std'].min(), df_test['pred_std'].mean())

df_test['smishing'] = df_test['pred_mean']

In [ ]:
df_test['smishing'].hist(bins=100, figsize=(20, 5))

In [ ]:
for c in pred_cols:
    print(c)
    display((df_test[c] * 10).astype(int).value_counts(sort=False))

In [ ]:
# 0     1504
# 1       11
# 2        6
# 3        6
# 4        2
# 5        3
# 6        2
# 9       39
# 10      53
(df_test['smishing'] * 10).astype(int).value_counts(sort=False)

In [ ]:
model_ts

In [ ]:
df_test[['smishing']].to_csv('submit/{}_nn.csv'.format(model_ts), index=True)
# df_test[['id', 'smishing', 'text']].sort_values('smishing', ascending=False).to_csv('{}_text.csv'.format(model_ts), index=False)
